In [32]:
import pickle
import gzip
import json
from glob import glob

def load_json(filename):
    with open(filename,mode="r") as f:
        return json.load(f)

def save_json(data,filename):
    with open(filename, mode="w") as f:
        json.dump(data,f)
    return

with gzip.open("object_color34_dict.pickle", 'r') as f:
    meta_dict = pickle.load(f)

In [33]:
f_devtest="simmc2_dials_dstc10_devtest.json"
f_dev="simmc2_dials_dstc10_dev.json"
f_train="simmc2_dials_dstc10_train.json"
files = [f_devtest,f_dev,f_train]

f_furn_meta="furniture_prefab_metadata_all.json"
f_fash_meta="fashion_prefab_metadata_all.json"
meta_files=[f_furn_meta,f_fash_meta]

dialogue_data=[]
for f in files:
    dialogue_data.extend(load_json(f)["dialogue_data"])
    
true_meta_dict={}
for f in meta_files:
    true_meta_dict.update(load_json(f))
    
# Get scene files (bbox + relationship)
scene_folder = "/home/nlplab9/Desktop/youngjae/youngjae/data/simmc2_scene_jsons_dstc10_public"
scene_list = sorted(glob(str(scene_folder + "/*_scene.json")))

In [34]:
def get_scene_objs(scene_name):
    matching = [s for s in scene_list if scene_name+"_scene.json" == s.split("/")[-1]]
    return load_json(matching[0])["scenes"][0]["objects"]

In [35]:
def get_color_objects(scene_name,idx_list):
    try:
        scene_objects=get_scene_objs(scene_name)
    except:
        print(scene_name)
        input()    
    objects={}
    for idx in idx_list:
        obj=[x for x in scene_objects if x['index'] == idx]
        if obj:
            obj=obj[0]
            try:
#                 objects[idx]=true_meta_dict[obj["prefab_path"]]["color"]
                # if color
                if len(true_meta_dict[obj["prefab_path"]]["color"].split(","))==1:
                    objects[idx]=true_meta_dict[obj["prefab_path"]]["color"]
                else:
                    objects[idx]="none"
            except:
#                 print(true_meta_dict[obj["prefab_path"]])
                objects[idx]="none"
                
    return objects

In [36]:
def get_type_objects(scene_name,idx_list):
    scene_objects=get_scene_objs(scene_name)
    objects={}
    for idx in idx_list:
        obj=[x for x in scene_objects if x['index'] == idx]
        if obj:
            obj=obj[0]
            try:
                objects[idx]=true_meta_dict[obj["prefab_path"]]["type"]
#                 # if color
#                 if len(true_meta_dict[obj["prefab_path"]]["color"].split(","))==1:
#                     objects[idx]=true_meta_dict[obj["prefab_path"]]["color"]
#                 else:
#                     objects[idx]="none"
            except:
#                 print(true_meta_dict[obj["prefab_path"]])
                objects[idx]="none"
                
    return objects

In [37]:
def get_patmat_objects(scene_name,idx_list):
    scene_objects=get_scene_objs(scene_name)
    objects={}
    for idx in idx_list:
        obj=[x for x in scene_objects if x['index'] == idx]
        if obj:
            obj=obj[0]
            try:
                if "pattern" in true_meta_dict[obj["prefab_path"]].keys():
                    objects[idx]=true_meta_dict[obj["prefab_path"]]["pattern"]
                else:
                    objects[idx]=true_meta_dict[obj["prefab_path"]]["material"]

            except:
#                 print(true_meta_dict[obj["prefab_path"]])
                objects[idx]="none"
                
    return objects

In [38]:
dialogue_data[0].keys()

dict_keys(['dialogue', 'dialogue_idx', 'domain', 'mentioned_object_ids', 'scene_ids'])

In [60]:
n_correct_color_obj = 0
n_predict_color_obj = 0
n_total_color_obj = 0
n_correct_type_obj = 0
n_predict_type_obj = 0
n_total_type_obj = 0
n_correct_patmat_obj = 0
n_predict_patmat_obj = 0
n_total_patmat_obj = 0

color_score_dict={}
patmat_score_dict={}
type_score_dict={}

color_total_dict={}
patmat_total_dict={}
type_total_dict={}

import collections
from collections import defaultdict
nested_dict = lambda: defaultdict(nested_dict)
color_comp_dict = nested_dict()

for d in dialogue_data:
    scenes = d["scene_ids"].values()
    for t in d["dialogue"]:
        idx_list =  t["system_transcript_annotated"]["act_attributes"]["objects"]
        turn_color_objects={}
        turn_type_objects={}
        turn_patmat_objects={}        
        for s in scenes:
            turn_color_objects.update(get_color_objects(s,idx_list))
            turn_type_objects.update(get_type_objects(s,idx_list))
            turn_patmat_objects.update(get_patmat_objects(s,idx_list))
        pred_turn_objects={}
        for s in scenes:
            pred_turn_objects.update(meta_dict[s])
# 
        # 0: type
        # 1: pattern material
        # 2: color
        for idx in idx_list:
            if pred_turn_objects[idx]:
                n_predict_type_obj += 1
                n_predict_color_obj += 1
                n_predict_patmat_obj += 1

            if turn_color_objects[idx]!="none":
                n_total_color_obj+=1
                if turn_color_objects[idx] not in color_total_dict:
                    color_total_dict[turn_color_objects[idx]]=0
                color_total_dict[turn_color_objects[idx]]+=1
#                 print(isinstance(color_comp_dict[turn_color_objects[idx]][pred_turn_objects[idx][2]],collections.defaultdict))
                if pred_turn_objects[idx] and isinstance(color_comp_dict[pred_turn_objects[idx][2]][turn_color_objects[idx]],collections.defaultdict):
                    color_comp_dict[pred_turn_objects[idx][2]][turn_color_objects[idx]]=0
                if pred_turn_objects[idx]:
                    color_comp_dict[pred_turn_objects[idx][2]][turn_color_objects[idx]]+=1
                if pred_turn_objects[idx] and turn_color_objects[idx]==pred_turn_objects[idx][2]:
                    n_correct_color_obj+=1
                    if turn_color_objects[idx] not in color_score_dict:
                        color_score_dict[turn_color_objects[idx]]=0
                    color_score_dict[turn_color_objects[idx]]+=1                    
                    
#             if turn_type_objects[idx]!="none":
#                 n_total_type_obj+=1
#                 if turn_type_objects[idx] not in type_total_dict:
#                     type_total_dict[turn_type_objects[idx]]=0
#                 type_total_dict[turn_type_objects[idx]]+=1                
#                 if pred_turn_objects[idx] and turn_type_objects[idx]==pred_turn_objects[idx][0]:
#                     n_correct_type_obj+=1
#                     if turn_type_objects[idx] not in type_score_dict:
#                         type_score_dict[turn_type_objects[idx]]=0
#                     type_score_dict[turn_type_objects[idx]]+=1                            
                    
#             if turn_patmat_objects[idx]!="none":
#                 n_total_patmat_obj+=1
#                 if turn_patmat_objects[idx] not in patmat_total_dict:
#                     patmat_total_dict[turn_patmat_objects[idx]]=0
#                 patmat_total_dict[turn_patmat_objects[idx]]+=1                 
#                 if pred_turn_objects[idx] and turn_patmat_objects[idx]==pred_turn_objects[idx][1]:
#                     n_correct_patmat_obj+=1      
#                     if turn_patmat_objects[idx] not in patmat_score_dict:
#                         patmat_score_dict[turn_patmat_objects[idx]]=0
#                     patmat_score_dict[turn_patmat_objects[idx]]+=1       

In [63]:
color_comp_dict
for k,v in color_comp_dict.items():
    print(k)
    print(sorted(v.items(),key=lambda x:-x[1]))

grey
[('grey', 4946), ('white', 1646), ('blue', 1023), ('brown', 976), ('black', 631), ('light grey', 300), ('wooden', 90), ('green', 76), ('dirty green', 48), ('violet', 47), ('dark grey', 42), ('black and white', 31), ('dark blue', 18), ('dark green', 14), ('light blue', 14), ('dirty grey', 13), ('light red', 6), ('yellow', 6), ('olive', 6), ('purple', 4)]
purple
[('purple', 578), ('pink', 360), ('maroon', 267), ('dark pink', 101), ('brown', 78), ('dark red', 76), ('grey', 72), ('black', 63), ('violet', 54), ('red', 23), ('olive', 22), ('beige', 17), ('blue', 16), ('light pink', 11), ('dark brown', 10), ('yellow', 10), ('light grey', 9), ('dark grey', 5), ('green', 4), ('dark blue', 2), ('white', 2)]
blue
[('brown', 3065), ('grey', 2857), ('black', 2609), ('blue', 2148), ('white', 1330), ('light grey', 746), ('wooden', 280), ('dark blue', 267), ('red', 261), ('green', 239), ('yellow', 158), ('beige', 155), ('light blue', 131), ('violet', 112), ('dark grey', 109), ('dark green', 96), 

In [67]:
pp_cand_dict = {
    "grey" : ["white"],
    "blue" : ["grey","brown","black"],
    "purple" : ["pink"],
    "brown" : ["blue"],
    "violet" : ["red","black"],
    "darkgreen" : ["brown"],
    "darkblue" : ["brown"],
    "yellow" : ["blue"],
    "dark red" : ["purple"],
    "light grey" : ["blue"],
    "maroon" : ["violet"],
    "golden" : ["light blue"],
    "beige" : ["blue"],
    "light orange" : ["blue"],
    "dark pink" : ["purple"],
    "light red" : ["grey"]
}

In [64]:
list("grey")

['g', 'r', 'e', 'y']

In [57]:
print(n_total_color_obj)
print(n_correct_color_obj)
print(n_total_type_obj)
print(n_correct_type_obj)
print(n_total_patmat_obj)
print(n_correct_patmat_obj)


53345
17974
0
0
0
0


In [58]:
print(n_correct_color_obj/n_total_color_obj)
print(n_correct_type_obj/n_total_type_obj)
print(n_correct_patmat_obj/n_total_patmat_obj)


0.33693879463867277


ZeroDivisionError: division by zero

In [ ]:
# pattern : 0.612
# type : 0.605
# color : 0.274

# color50 : 0.254

In [ ]:
# color 
# total: 53281
# correct: 14622

# color50
# total: 53281
# correct: 13555


In [56]:
cold={}
for i in color_score_dict.keys():
    print(f"{i}\t{round(color_score_dict[i]/color_total_dict[i],2)}")
    cold[i]=[round(color_score_dict[i]/color_total_dict[i],2)]
print()
patd={}
for i in patmat_score_dict.keys():
    patd[i]=[round(patmat_score_dict[i]/patmat_total_dict[i],2)]
    print(f"{i}\t{round(patmat_score_dict[i]/patmat_total_dict[i],2)}")
print()
typd={}
for i in type_score_dict.keys():
    typd[i]=[round(type_score_dict[i]/type_total_dict[i],2)]
    print(f"{i}\t{round(type_score_dict[i]/type_total_dict[i],2)}")
print()
print(cold)

    
print()
print(color_score_dict)
print(patmat_score_dict)
print(type_score_dict)

print(color_total_dict)
print(patmat_total_dict)
print(type_total_dict)

grey	0.52
black	0.53
green	0.59
violet	0.08
blue	0.36
purple	0.62
pink	0.1
dark grey	0.15
white	0.41
red	0.29
dark green	0.14
wooden	0.33
light grey	0.0
dark blue	0.01
dirty grey	0.11
brown	0.01



{'grey': [0.52], 'black': [0.53], 'green': [0.59], 'violet': [0.08], 'blue': [0.36], 'purple': [0.62], 'pink': [0.1], 'dark grey': [0.15], 'white': [0.41], 'red': [0.29], 'dark green': [0.14], 'wooden': [0.33], 'light grey': [0.0], 'dark blue': [0.01], 'dirty grey': [0.11], 'brown': [0.01]}

{'grey': 4946, 'black': 5111, 'green': 1519, 'violet': 51, 'blue': 2148, 'purple': 578, 'pink': 69, 'dark grey': 49, 'white': 2224, 'red': 349, 'dark green': 70, 'wooden': 793, 'light grey': 3, 'dark blue': 8, 'dirty grey': 5, 'brown': 51}
{}
{}
{'grey': 9471, 'maroon': 638, 'dark brown': 159, 'brown': 7235, 'black': 9565, 'dark blue': 895, 'light grey': 2108, 'green': 2567, 'beige': 333, 'dark green': 516, 'golden': 107, 'purple': 935, 'blue': 5897, 'pink': 662, 'dark grey': 332, 'violet': 616, 'red': 1